###  importing relevant libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from random import random
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix
#time.sleep(3600)

### Discovering Labels

In [3]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    df["Label"]=df["Label"].replace(renamer)
    df=df.dropna()
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [4]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [5]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [6]:
find_the_way("small",".csv")

['small\\uk-vpn.csv', 'small\\uk.csv', 'small\\us-vpn.csv', 'small\\us.csv']

In [7]:
renamer={}

In [8]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['eth.src']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [9]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):

    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [10]:


def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]


In [11]:
def MLCV(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        fold=2
        repetition=12
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):


            rnd = random()
            
            kfold = sklearn.model_selection.KFold(n_splits=fold, shuffle=True, random_state=int(rnd*100))  
            cv=0
            df = pd.read_csv(loop1,usecols=cols)#,header=None )
            ##df = df.reset_index(drop=True)
            df=df.fillna(0)
            #macs=df["eth.src"]
            #del df["eth.src"]
            #del df["MAC"] # if dataset has MAC colomn please uncomment this line
            X =df[df.columns[0:-1]]
            X=np.array(X)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y=df[df.columns[-1]].cat.codes  
            X.shape

            for train_index, test_index in kfold.split(X):
                #try:
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index] 
                #m_train, m_test = macs[train_index], macs[test_index] 

            
            

                results_y=[]
                cv+=1
                results_y.append(y_test)
    
    
                #machine learning algorithm is applied in this section
                clf = ml_list[ii]#choose algorithm from ml_list dictionary
                second=time.time()
                clf.fit(X_train, y_train)
                train_time=(float((time.time()-second)) )
                second=time.time()
                predict =clf.predict(X_test)
                test_time=(float((time.time()-second)) )
                if step==1:
                    altime=0
                    line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                else:
                    predict,altime=merged(m_test,predict,step,mixed)
                    line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                    
                    
                lines.append(line)
                df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
                if cm.empty:
                    cm =df_cm
                else:
                    cm = cm.add(df_cm, fill_value=0)
                
        class_based_results=class_based_results/(repetition*fold)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()

        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//(repetition*fold)
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



In [12]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        repetition=25
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            df=df.dropna()
            try:
                m_train=df["eth.src"]
                del df["eth.src"]
            except:pass
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df=df.dropna()
            try:
                m_test=df["eth.src"]
                del df["eth.src"]
            except:pass
            
            df["Label"]=df["Label"].replace(renamer)
            #df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes


            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )

            altime=0
            line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)

            
                
            lines.append(line)
            df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



# Machine learning applications 

# Aalto Dataset

In [13]:
feature=['ACK Flag Cnt',
 'Active Max',
 'Active Mean',
 'Active Std',
 'Bwd Header Len',
 'Bwd IAT Min',
 'Bwd IAT Tot',
 'Bwd Pkt Len Max',
 'Bwd Pkt Len Mean',
 'Bwd Pkt Len Min',
 'FIN Flag Cnt',
 'Flow Byts/s',
 'Flow Duration',
 'Flow IAT Max',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Fwd IAT Mean',
 'Fwd IAT Min',
 'Fwd IAT Std',
 'Fwd IAT Tot',
 'Fwd Pkt Len Max',
 'Fwd Pkt Len Std',
 'Fwd Pkts/s',
 'Idle Max',
 'Idle Mean',
 'Idle Std',
 'Init Bwd Win Byts',
 'Pkt Len Max',
 'Pkt Len Min',
 'Pkt Len Var',
 'Pkt Size Avg',
 'Protocol',
 'Src Port',
 'Subflow Bwd Byts',
 'Tot Fwd Pkts',
 'TotLen Bwd Pkts',
 'Label']

In [14]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


# Cross-Validation

In [15]:

ml_list={#"DT":DecisionTreeClassifier()
    "RF":RandomForestClassifier(bootstrap=False,criterion="gini",max_depth=17,max_features=10,min_samples_split=7,n_estimators=188)}

In [16]:
file_list={"UKVPN":'small/uk-vpn.csv',
           "UK":'small/uk.csv',
           "USVPN":'small/us-vpn.csv',
           "US":'small/us.csv'}

### Normal

In [17]:
savehere="results/cv/1/"
folder(savehere)
step=1
mixed=0
for file in file_list:
    print(file)
    train=file_list[file]
    test=file_list[file]
    output_csv=f"{savehere}{file}_ML_.csv"
    target_names=target_name(file_list[file])
    MLCV(train,test,output_csv,feature,step,mixed,file)   


UKVPN
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN        0     1  RF    0.899    0.834   0.867  0.834  0.84   0.886      0   36.497     0.825          0
 1  UKVPN        0     2  RF    0.901    0.848   0.877  0.848  0.858  0.888      0   37.153     0.909          0
 2  UKVPN        1     1  RF    0.903    0.865   0.87   0.865  0.855  0.891      0   37.473     0.715          0
 3  UKVPN        1     2  RF    0.897    0.826   0.89   0.826  0.841  0.884      0   35.841     0.697          0
 4  UKVPN        2     1  RF    0.902    0.851   0.898  0.851  0.868  0.889      0   36.042     0.862          0
 5  UKVPN        2     2  RF    0.901    0.848   0.893  0.848  0.864  0.888      0   36.294     0.724          0
 6  UKVPN        3     1  RF    0.903    0.845   0.892  0.845  0.862  0.89       0   35.93

# Session vs Session

In [18]:
file_list={"UKVPN@UK":['small/uk-vpn.csv','small/uk.csv'],
           "UK@UKVPN":['small/uk.csv','small/uk-vpn.csv'],
  "USVPN@US":['small/us-vpn.csv','small/us.csv'],
           "US@USVPN":['small/us.csv','small/us-vpn.csv']}

In [19]:
### Normal

In [20]:
say=0
savehere="results/ss/1/"
folder(savehere)


for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@UK
    Dataset      T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  ---------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@UK     0     1  RF    0.466    0.616   0.664  0.616  0.565  0.441      0   83.412     1.641          0
 1  UKVPN@UK     1     2  RF    0.463    0.609   0.645  0.609  0.552  0.438      0   83.751     1.62           0
 2  UKVPN@UK     2     3  RF    0.458    0.608   0.655  0.608  0.553  0.433      0   83.49      1.604          0
 3  UKVPN@UK     3     4  RF    0.459    0.603   0.645  0.603  0.548  0.433      0   83.597     1.661          0
 4  UKVPN@UK     4     5  RF    0.47     0.625   0.654  0.625  0.562  0.445      0   83.421     1.622          0
 5  UKVPN@UK     5     6  RF    0.457    0.604   0.653  0.604  0.551  0.432      0   83.952     1.619          0
 6  UKVPN@UK     6     7  RF    0.46     0.6     0.658  0.6    0.553  0.435      0   8

# Dataset Vs Dataset

In [21]:
file_list={"UKVPN@USVPN":['small/uk-vpn.csv','small/us-vpn.csv'],
"UKVPN@US":['small/uk-vpn.csv','small/us.csv'],
"UK@USVPN":['small/uk.csv','small/us-vpn.csv'],
"UK@US":['small/uk.csv','small/us.csv'],
"USVPN@UKVPN":['small/us-vpn.csv','small/uk-vpn.csv'],
"USVPN@UK":['small/us-vpn.csv','small/uk.csv'],
"US@UKVPN":['small/us.csv','small/uk-vpn.csv'],
"US@UK":['small/us.csv','small/uk.csv']}

### Normal

In [22]:
say=0
savehere="results/dd/1/"
folder(savehere)
for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{savehere}/ML@{ii}.csv"
        print(ii)
    
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,ii)   


UKVPN@USVPN
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  UKVPN@USVPN    0     1  RF    0.453    0.522   0.536  0.522  0.45   0.425      0   81.785     1.747          0
 1  UKVPN@USVPN    1     2  RF    0.451    0.522   0.539  0.522  0.451  0.423      0   81.546     1.89           0
 2  UKVPN@USVPN    2     3  RF    0.466    0.528   0.544  0.528  0.458  0.439      0   81.89      1.763          0
 3  UKVPN@USVPN    3     4  RF    0.449    0.521   0.534  0.521  0.444  0.42       0   82.204     1.781          0
 4  UKVPN@USVPN    4     5  RF    0.451    0.522   0.544  0.522  0.451  0.423      0   81.903     1.767          0
 5  UKVPN@USVPN    5     6  RF    0.451    0.519   0.54   0.519  0.45   0.423      0   81.857     1.782          0
 6  UKVPN@USVPN    6     7  RF    0.455    0.526   0.535  0.526  0.

In [ ]:
!shutdown /s /t 360